# Stream data

When working with large serialized objects, it is often inefficient to load entire files into memory.

Here, we show how to subset an `AnnData` stored in the cloud.

In [ ]:
import lamindb as ln

In [ ]:
ln.track()

Check the configured storage:

In [ ]:
ln.setup.settings.storage.root

Register a file:

In [ ]:
file = ln.File("s3://lamindb-ci/lndb-storage/pbmc68k.h5ad")
file = ln.add(file)

Get its backed cloud representation:

In [ ]:
adata = file.backed()

Inspect its metadata:

In [ ]:
adata.obs.head()

In [ ]:
adata.obs.cell_type.value_counts()

Construct a subsetter based on the metadata:

In [ ]:
obs = file.subsetter()
subset_obs = obs.cell_type.isin(["Dendritic cells", "CD14+ Monocytes"]) & (
    obs.percent_mito <= 0.05
)

In [ ]:
adata_subset = file.stream(subset_obs=subset_obs)

In [ ]:
adata_subset

In [ ]:
adata_subset.obs.cell_type.value_counts()

It is also possible to access AnnData objects' attributes and subset them directly through `file.backed()` withouth loading the full objects into memory:

In [ ]:
adata

Note that the object above is an AnnDataAccessor object, not an AnnData object

Check the reference to `.X`:

In [ ]:
adata.X

Get a subset of the object, attributes are loaded only on explicit access:

In [ ]:
obs_idx = adata.obs.cell_type.isin(["Dendritic cells", "CD14+ Monocytes"]) & (
    adata.obs.percent_mito <= 0.05
)
adata_subset = adata[obs_idx]

In [ ]:
adata_subset

In [ ]:
adata_subset.obs.cell_type.value_counts()

You can do the same with a zarr object:

In [ ]:
file = ln.add(ln.File("s3://lamindb-ci/lndb-storage/pbmc68k.zarr"))
print(file.backed().obs.head())
adata_subset = file.stream(subset_obs=subset_obs)
adata_subset.obs.cell_type.value_counts()